# **Jupyter Notebook interativo para cálculo de parâmetros hemodinâmicos de vídeos de coração do inseto**



<font size=3>Autores: [Felipe Hiroshi Kano Inazumi](mailto:f215696@dac.unicamp.br),
[Nelly Catherine Barbosa Calderon](mailto:n160942@dac.unicamp.br), 
[Adriano Santana](mailto:adriano.rsantana@gmail.com),
[Rosana Almada Bassani](mailto:arbassani@unicamp.br), 
[José Wilson Magalhães Bassani](bassani@unicamp.br)

Execute a célula abaixo, em seguida será necessário especificar o local do arquivo de vídeo e a distância entre cada pixel da imagem em **$μm$**


Por exemplo:

`Video Path: "C:/Videos/meu_video.avi"`

`Distance between pixels: 3.5 `


In [ ]:
import exec_video
exec_video.menu_start()

# Referências

1.	Virtanen P, Gommers R, Oliphant TE, Haberland M, Reddy T, Cournapeau D, et al. SciPy 1.0: Fundamental Algorithms for Scientific Computing in Python. Nat Methods. 2020;17:261–72. 
2.	Harris CR, Millman KJ, der Walt SJ van, Gommers R, Virtanen P, David Cournapeau, et al. Array programming with NumPy. Nature [Internet]. 2020 set;585(7825):357–62. Available from: https://doi.org/10.1038/s41586-020-2649-2
3.	pandas development team T. pandas-dev/pandas: Pandas [Internet]. Zenodo; 2020. Available from: https://doi.org/10.5281/zenodo.3509134
4.	Bradski G. The OpenCV Library. Dr Dobb’s Journal of Software Tools. 2000; 
5.	Hunter JD. Matplotlib: A 2D graphics environment. Comput Sci Eng. 2007;9(3):90–5. 